In [3]:
#scikit learn ensembe workflow for binary probability
# https://www.kaggle.com/the1owl/can-rf-challenge-xgboost
import time; start_time = time.time()
import numpy as np
import pandas as pd
from sklearn import ensemble
import xgboost as xgb
from sklearn.metrics import log_loss, make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
import random; random.seed(2016)

In [10]:
%matplotlib inline

pd.set_option("display.max_columns", 200) # 表示カラムの最大値
pd.set_option('display.max_rows', 50) # 表示行数の最大値
pd.set_option('display.max_colwidth', 30)
pd.set_option("display.float_format", '{:.4f}'.format) # 小数点以下の有効数字

In [11]:
# jupyterの幅を自動調整
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [6]:
def fill_nan_null(val):
    ret_fill_nan_null = 0.0
    if val == True:
        ret_fill_nan_null = 1.0
    return ret_fill_nan_null

In [4]:
train = pd.read_csv('../01.data/train.csv')
test = pd.read_csv('../01.data/test.csv')
num_train = train.shape[0]

In [7]:
y_train = train['target']
train = train.drop(['target'],axis=1)
id_test = test['ID']

In [8]:
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all['null_count'] = df_all.isnull().sum(axis=1).tolist()
df_all_temp = df_all['ID']
df_all = df_all.drop(['ID'],axis=1)
df_data_types = df_all.dtypes[:] #{'object':0,'int64':0,'float64':0,'datetime64':0}
d_col_drops = []

In [12]:
df_all.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,null_count
0,1.3357,8.7275,C,3.9210,7.9153,2.5993,3.1769,0.0129,10.0000,0.5033,16.4341,6.0857,2.8668,11.6364,1.3550,8.5714,3.6704,0.1067,0.1489,18.8693,7.7309,XDX,-0.0000,C,0.1394,1.7208,3.3935,0.5901,8.8809,C,A,1.0830,1.0108,7.2701,8.3755,11.3266,0.4545,0,4.0121,7.7115,7.6534,12.7076,2.0155,10.4983,9.8487,0.1136,C,12.1717,8.0866,0.8994,7.2778,G,16.7480,0.0371,1.2996,DI,3.9711,0.5298,10.8910,1.5884,15.8582,1,0.1535,6.3632,18.3039,C,9.3141,15.2318,17.1429,11.7845,F,1,1.6150,B,D,2.2309,7.2924,8.5714,E,3.0000,7.5283,8.8616,0.6498,1.2996,1.7073,0.8664,9.5518,3.3213,0.0957,0.9053,A,0.4423,5.8140,3.5177,0.4620,7.4368,5.4545,8.8774,1.1913,19.4702,8.3892,2.7574,4.3743,1.5740,0.0073,12.5792,E,2.3827,3.9309,B,0.4332,O,NaN,15.6349,2.8571,1.9512,6.5920,5.9091,-0.0000,1.0596,0.8036,8.0000,1.9898,0.0358,AU,1.8041,3.1137,2.0243,0,0.6364,2.8571,1
1,nan,nan,C,nan,9.1913,nan,nan,2.3016,nan,1.3129,nan,6.5076,nan,11.6364,nan,nan,nan,nan,nan,nan,6.7631,GUV,nan,C,3.0561,nan,nan,nan,nan,C,A,nan,nan,3.6151,nan,14.5795,nan,0,nan,14.3058,nan,nan,nan,nan,nan,2.4500,E,nan,nan,1.3792,nan,G,nan,1.1295,nan,DY,nan,nan,nan,nan,nan,2,2.5447,nan,nan,A,nan,nan,nan,12.0534,F,2,nan,B,D,nan,nan,nan,D,nan,7.2777,3.4307,nan,nan,nan,nan,9.8480,nan,2.6786,nan,B,nan,nan,nan,nan,nan,nan,8.3040,nan,nan,nan,nan,nan,nan,1.5053,nan,B,1.8254,4.2479,A,nan,U,G,10.3080,nan,nan,10.5954,nan,nan,nan,nan,nan,nan,0.5989,AF,nan,nan,1.9578,0,nan,nan,81
2,0.9439,5.3101,C,4.4110,5.3262,3.9796,3.9286,0.0196,12.6667,0.7659,14.7561,6.3847,2.5056,9.6035,1.9841,5.8824,3.1708,0.2445,0.1443,17.9523,5.2450,FQ,-0.0000,E,0.1140,2.2449,5.3061,0.8360,7.5000,NaN,A,1.4541,1.7347,4.0439,7.9592,12.7305,0.2597,0,7.3790,13.0772,6.1735,12.3469,2.9268,8.8976,5.3438,0.1260,C,12.7113,6.8367,0.6045,9.6376,F,15.1020,0.0856,0.7653,AS,4.0306,4.2775,9.1055,2.1514,16.0756,1,0.1236,5.5179,16.3772,A,8.3673,11.0405,5.8824,8.4607,B,3,2.4136,B,B,1.9640,5.9184,11.7647,E,3.3333,10.1944,8.2662,1.5306,1.5306,2.4299,1.0714,8.4475,3.3673,0.1114,0.8114,G,0.2715,5.1566,4.2149,0.3097,5.6633,5.9740,11.5889,0.8418,15.4913,5.8794,3.2928,5.9245,1.6684,0.0083,11.6706,C,1.3758,1.1842,B,3.3673,S,NaN,11.2056,12.9412,3.1293,3.4789,6.2338,-0.0000,2.1387,2.2388,9.3333,2.4776,0.0135,AE,1.7737,3.9222,1.1205,2,0.8831,1.1765,2
3,0.7974,8.3048,C,4.2259,11.6274,2.0977,1.9875,0.1719,8.9655,6.5427,16.3475,9.6467,3.9033,14.0947,1.9450,5.5172,3.6108,1.2241,0.2316,18.3764,7.5171,ACUE,-0.0000,D,0.1488,1.3083,2.3036,8.9267,8.8745,C,B,1.5876,1.6667,8.7035,8.8985,11.3028,0.4337,0,0.2873,11.5230,7.9310,12.9358,1.4709,12.7086,9.6708,0.1084,C,12.1949,8.5920,3.3292,4.7804,H,16.6217,0.1397,1.1782,BW,3.9655,1.7321,11.7779,1.2292,15.9274,1,0.1403,6.2930,17.0116,A,9.7031,18.5681,9.4253,13.5947,F,2,2.2725,B,D,2.1882,8.2136,13.4483,B,1.9473,4.7979,13.3158,1.6810,1.3793,1.5870,1.2428,10.7471,1.4080,0.0391,1.0424,B,0.7639,5.4989,3.4239,0.8325,7.3755,6.7470,6.9420,1.3346,18.2564,8.5073,2.5031,4.8722,2.5737,0.1140,12.5543,B,2.2308,1.9901,B,2.6437,J,NaN,13.7777,10.5747,1.5111,4.9496,7.1807,0.5655,1.1663,1.9565,7.0183,1.8128,0.0023,CJ,1.4152,2.9544,1.9908,1,1.6771,1.0345,1
4,nan,nan,C,nan,nan,nan,nan,nan,nan,1.0503,nan,6.3201,nan,10.9911,nan,nan,nan,nan,nan,nan,6.4146,HIT,nan,E,nan,nan,nan,nan,nan,NaN,A,nan,nan,6.0832,nan,nan,nan,0,nan,10.1389,nan,nan,nan,nan,nan,nan,I,nan,nan,1.3645,nan,H,nan,nan,nan,NaN,nan,nan,nan,nan,nan,1,nan,nan,nan,C,nan,nan,nan,nan,F,1,nan,B,D,nan,nan,nan,C,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,G,nan,n

In [13]:
for i in range(len(df_data_types)):
    df_all[str(df_data_types.index[i])+'_nan_'] = df_all[str(df_data_types.index[i])].map(lambda x:fill_nan_null(pd.isnull(x)))
df_all = df_all.fillna(-9999)
#df_all = df_all.replace(0, -9999)

In [14]:
df_all.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,...,v33_nan_,v34_nan_,v35_nan_,v36_nan_,v37_nan_,v38_nan_,v39_nan_,v40_nan_,v41_nan_,v42_nan_,v43_nan_,v44_nan_,v45_nan_,v46_nan_,v47_nan_,v48_nan_,v49_nan_,v50_nan_,v51_nan_,v52_nan_,v53_nan_,v54_nan_,v55_nan_,v56_nan_,v57_nan_,v58_nan_,v59_nan_,v60_nan_,v61_nan_,v62_nan_,v63_nan_,v64_nan_,v65_nan_,v66_nan_,v67_nan_,v68_nan_,v69_nan_,v70_nan_,v71_nan_,v72_nan_,v73_nan_,v74_nan_,v75_nan_,v76_nan_,v77_nan_,v78_nan_,v79_nan_,v80_nan_,v81_nan_,v82_nan_,v83_nan_,v84_nan_,v85_nan_,v86_nan_,v87_nan_,v88_nan_,v89_nan_,v90_nan_,v91_nan_,v92_nan_,v93_nan_,v94_nan_,v95_nan_,v96_nan_,v97_nan_,v98_nan_,v99_nan_,v100_nan_,v101_nan_,v102_nan_,v103_nan_,v104_nan_,v105_nan_,v106_nan_,v107_nan_,v108_nan_,v109_nan_,v110_nan_,v111_nan_,v112_nan_,v113_nan_,v114_nan_,v115_nan_,v116_nan_,v117_nan_,v118_nan_,v119_nan_,v120_nan_,v121_nan_,v122_nan_,v123_nan_,v124_nan_,v125_nan_,v126_nan_,v127_nan_,v128_nan_,v129_nan_,v130_nan_,v131_nan_,null_count_nan_
0,1.3357,8.7275,C,3.9210,7.9153,2.5993,3.1769,0.0129,10.0000,0.5033,16.4341,6.0857,2.8668,11.6364,1.3550,8.5714,3.6704,0.1067,0.1489,18.8693,7.7309,XDX,-0.0000,C,0.1394,1.7208,3.3935,0.5901,8.8809,C,A,1.0830,1.0108,7.2701,8.3755,11.3266,0.4545,0,4.0121,7.7115,7.6534,12.7076,2.0155,10.4983,9.8487,0.1136,C,12.1717,8.0866,0.8994,7.2778,G,16.7480,0.0371,1.2996,DI,3.9711,0.5298,10.8910,1.5884,15.8582,1,0.1535,6.3632,18.3039,C,9.3141,15.2318,17.1429,11.7845,F,1,1.6150,B,D,2.2309,7.2924,8.5714,E,3.0000,7.5283,8.8616,0.6498,1.2996,1.7073,0.8664,9.5518,3.3213,0.0957,0.9053,A,0.4423,5.8140,3.5177,0.4620,7.4368,5.4545,8.8774,1.1913,19.4702,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,-9999.0000,-9999.0000,C,-9999.0000,9.1913,-9999.0000,-9999.0000,2.3016,-9999.0000,1.3129,-9999.0000,6.5076,-9999.0000,11.6364,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,6.7631,GUV,-9999.0000,C,3.0561,-9999.0000,-9999.0000,-9999.0000,-9999.0000,C,A,-9999.0000,-9999.0000,3.6151,-9999.0000,14.5795,-9999.0000,0,-9999.0000,14.3058,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,2.4500,E,-9999.0000,-9999.0000,1.3792,-9999.0000,G,-9999.0000,1.1295,-9999.0000,DY,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,2,2.5447,-9999.0000,-9999.0000,A,-9999.0000,-9999.0000,-9999.0000,12.0534,F,2,-9999.0000,B,D,-9999.0000,-9999.0000,-9999.0000,D,-9999.0000,7.2777,3.4307,-9999.0000,-9999.0000,-9999.0000,-9999.0000,9.8480,-9999.0000,2.6786,-9999.0000,B,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,8.3040,-9999.0000,-9999.0000,...,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000,0.0000,1.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000,0.0000,1.0000,1.0000,0.0000,1.0000,1.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,1.0000,0.0000,0.0000,1.0000,1.0000,1.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.

In [15]:
for i in range(len(df_data_types)):
    if str(df_data_types[i])=='object':
        df_u = pd.unique(df_all[str(df_data_types.index[i])].ravel())
        print("Column: ", str(df_data_types.index[i]), " Length: ", len(df_u))
        d={}
        j = 1000
        for s in df_u:
            d[str(s)]=j
            j+=5
        df_all[str(df_data_types.index[i])+'_vect_'] = df_all[str(df_data_types.index[i])].map(lambda x:d[str(x)])
        d_col_drops.append(str(df_data_types.index[i]))
        if len(df_u)<150:
            dummies = pd.get_dummies(df_all[str(df_data_types.index[i])]).rename(columns=lambda x: str(df_data_types.index[i]) + '_' + str(x))
            df_all_temp = pd.concat([df_all_temp, dummies], axis=1)

Column:  v3  Length:  4
Column:  v22  Length:  23420
Column:  v24  Length:  5
Column:  v30  Length:  8
Column:  v31  Length:  4
Column:  v47  Length:  10
Column:  v52  Length:  13
Column:  v56  Length:  131
Column:  v66  Length:  3
Column:  v71  Length:  12
Column:  v74  Length:  3
Column:  v75  Length:  4
Column:  v79  Length:  18
Column:  v91  Length:  8
Column:  v107  Length:  8
Column:  v110  Length:  3
Column:  v112  Length:  23
Column:  v113  Length:  38
Column:  v125  Length:  91


In [16]:
df_all_temp = df_all_temp.drop(['ID'],axis=1)
df_all = pd.concat([df_all, df_all_temp], axis=1)
print(len(df_all), len(df_all.columns))

228714 669


In [17]:
train = df_all.iloc[:num_train]
test = df_all.iloc[num_train:]
train = train.drop(d_col_drops,axis=1)
test = test.drop(d_col_drops,axis=1)

In [18]:
train.head()

,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v23,v25,v26,v27,v28,v29,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v48,v49,v50,v51,v53,v54,v55,v57,v58,v59,v60,v61,v62,v63,v64,v65,v67,v68,v69,v70,v72,v73,v76,v77,v78,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v108,v109,v111,v114,v115,v116,v117,v118,...,v113_R,v113_S,v113_T,v113_U,v113_V,v113_W,v113_X,v113_Y,v113_Z,v125_-9999,v125_A,v125_AA,v125_AB,v125_AC,v125_AD,v125_AE,v125_AF,v125_AG,v125_AH,v125_AI,v125_AJ,v125_AK,v125_AL,v125_AM,v125_AN,v125_AO,v125_AP,v125_AQ,v125_AR,v125_AS,v125_AT,v125_AU,v125_AV,v125_AW,v125_AX,v125_AY,v125_AZ,v125_B,v125_BA,v125_BB,v125_BC,v125_BD,v125_BE,v125_BF,v125_BG,v125_BH,v125_BI,v125_BJ,v125_BK,v125_BL,v125_BM,v125_BN,v125_BO,v125_BP,v125_BQ,v125_BR,v125_BS,v125_BT,v125_BU,v125_BV,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z
0,1.3357,8.7275,3.9210,7.9153,2.5993,3.1769,0.0129,10.0000,0.5033,16.4341,6.0857,2.8668,11.6364,1.3550,8.5714,3.6704,0.1067,0.1489,18.8693,7.7309,-0.0000,0.1394,1.7208,3.3935,0.5901,8.8809,1.0830,1.0108,7.2701,8.3755,11.3266,0.4545,0,4.0121,7.7115,7.6534,12.7076,2.0155,10.4983,9.8487,0.1136,12.1717,8.0866,0.8994,7.2778,16.7480,0.0371,1.2996,3.9711,0.5298,10.8910,1.5884,15.8582,1,0.1535,6.3632,18.3039,9.3141,15.2318,17.1429,11.7845,1,1.6150,2.2309,7.2924,8.5714,3.0000,7.5283,8.8616,0.6498,1.2996,1.7073,0.8664,9.5518,3.3213,0.0957,0.9053,0.4423,5.8140,3.5177,0.4620,7.4368,5.4545,8.8774,1.1913,19.4702,8.3892,2.7574,4.3743,1.5740,0.0073,12.5792,2.3827,3.9309,0.4332,15.6349,2.8571,1.9512,6.5920,5.9091,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-9999.0000,-9999.0000,-9999.0000,9.1913,-9999.0000,-9999.0000,2.3016,-9999.0000,1.3129,-9999.0000,6.5076,-9999.0000,11.6364,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,6.7631,-9999.0000,3.0561,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,3.6151,-9999.0000,14.5795,-9999.0000,0,-9999.0000,14.3058,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,2.4500,-9999.0000,-9999.0000,1.3792,-9999.0000,-9999.0000,1.1295,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,2,2.5447,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,12.0534,2,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,7.2777,3.4307,-9999.0000,-9999.0000,-9999.0000,-9999.0000,9.8480,-9999.0000,2.6786,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,8.3040,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,1.5053,-9999.0000,1.8254,4.2479,-9999.0000,10.3080,-9999.0000,-9999.0000,10.5954,-9999.0000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.9439,5.3101,4.4110,5.3262,3.9796,3.9286,0.0196,12.6667,0.7659,14.7561,6.3847,2.5056,9.6035,1.9841,5.8824,3.1708,0.2445,0.1443,17.9523,5.2450,-0.0000,0.1140,2.2449,5.3061,0.8360,7.5000,1.4541,1.7347,4.0439,7.9592,12.7305,0.2597,0,7.3790,13.0772,6.1735,12.3469,2.9268,8.8976,5.3438,0.1260,12.7113,6.8367,0.6045,9.6376,15.1020,0.0856,0.7653,4.0306,4.2775,9.1055,2.1514,16.0756,1,0.1236,5.5179,16.3772,8.3673,11.0405,5.8824,8.4607,3,2.4136,1.9640,5.9184,11.7647,3.3333,10.1944,8.2662,1.5306,1.5306,2.4299,1.0714,8.4475,3.3673,0.1114,0.8114,0.2715,5.1566,4.2149,0.3097,5.6633,5.9740,11.5889,0.8418,15.4913,5.8794,3.2928,5.9245,1.6684,0.0083,11.6706,1.3758,1.1842,3.3673,11.2056,12.9412,3.1293,3.4789,6.2338,...,0,0,0,0,0,0,0,0